In [14]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
import pickle
# import os

def predict(file_path):
  df_file = pd.read_csv(file_path,header=1,skip_blank_lines=True,skiprows =[0,1,2,3])
  df_file_info = pd.read_csv(file_path,header=None,nrows=4,skip_blank_lines=True)
  activity = df_file_info.iloc[2,1].lower()
  activity = 'run' if 'run' in activity else 'walk'
  actucal_steps = df_file_info.iloc[3,1]

  norm = []
  cols = df_file.columns
  for i, row in df_file.iterrows():
    x = float(row[cols[1]])
    y = float(row[cols[2]])
    z = float(row[cols[3]])
    norm.append((x**2 + y**2 + z**2) ** 0.5)
  norm = np.array(norm)
  df_file['norm'] = norm
  features = []
  cols = df_file.columns
  for col_name in [cols[0],cols[3],cols[4]]:
    vals = df_file[col_name].values
    vals_lag = vals-(np.append(vals[1:],0))
    vals_2lag = vals-(np.append(vals[2:],[0,0]))
    features.append(vals.mean())
    features.append(vals.std())
    features.append(vals.max())
    features.append(vals.min())
    features.append(vals_lag.mean())
    features.append(vals_lag.std())
    features.append(vals_lag.max())
    features.append(vals_lag.min())
    features.append(vals_2lag.mean())
    features.append(vals_2lag.std())
    features.append(vals_2lag.max())
    features.append(vals_2lag.min())
  fft = np.fft.fft(norm)
  time_vals = df_file[cols[0]].values
  dt = time_vals[1]-time_vals[0]
  n = len(norm)
  frequencies = np.fft.fftfreq(n, dt)
  features.append(len(time_vals))
  features.append(frequencies.mean())
  features.append(frequencies.std())
  features.append(frequencies.max())
  features.append(frequencies.min())
  features = np.array(features)
  X=[]
  X.append(features)
  X = np.stack(X)
  X = np.nan_to_num(X, copy=True, nan=0.0, posinf=None, neginf=None)
  activity_model = pickle.load(open('random_forest_model.pkl', 'rb'))
  activity_pred = activity_model.predict(X)
  if activity_pred == 1:
    run_model = pickle.load(open('steps_run_model.pkl', 'rb'))
    predictions = run_model.predict(X).round()
    activity_name = "Running"
  else:
    walk_model = pickle.load(open('steps_walk_model.pkl', 'rb'))
    predictions = walk_model.predict(X).round()
    activity_name = "Walking"
  print(f"Predicted activity is: {activity_name}, steps counted: {int(predictions[0])}")
  return [int(predictions[0]),activity_pred[0]]



In [15]:
# directory_path = "/content/data/"
# directory_files = os.listdir(directory_path)
# for path in directory_files:
#   predictions = predict(directory_path+path)
#   [steps_pred,activity_pred] = predictions
#   print(path)
#   print(steps_pred)
#   print(activity_pred)

Predicted activity is: Walking, steps counted: 88
walk_5_3.csv
88
0
Predicted activity is: Running, steps counted: 111
run_2_3.csv
111
1
Predicted activity is: Running, steps counted: 120
run_1_3.csv
120
1
Predicted activity is: Running, steps counted: 110
run_3_3.csv
110
1
Predicted activity is: Running, steps counted: 66
run_5_2.csv
66
1
Predicted activity is: Running, steps counted: 57
run_4_3.csv
57
1
Predicted activity is: Walking, steps counted: 90
walk_4_3.csv
90
0
Predicted activity is: Walking, steps counted: 53
walk_1_2.csv
53
0
Predicted activity is: Walking, steps counted: 50
walk_2_2.csv
50
0
Predicted activity is: Walking, steps counted: 90
walk_3_2.csv
90
0
